In [2]:
import numpy as np
import pandas as pd

In [3]:
authors  = pd.DataFrame([[1, 'Тургенев'],  [2, 'Чехов'], [3, 'Островский']], columns = ['author_id', 'author_name'])
print(authors)

   author_id author_name
0          1    Тургенев
1          2       Чехов
2          3  Островский


In [4]:
data = np.array([[1, 1, 1, 2, 2, 3, 3],
            ['Отцы и дети', 'Рудин', 'Дворянское гнездо', 'Толстый и тонкий', 'Дама с собачкой', 'Гроза', 'Таланты и поклонники'],
            [450, 300, 350, 500, 450, 370, 290]
    ]).T.tolist()
book = pd.DataFrame(data, columns = ['author_id', 'book_title', 'price'])
print(book)

  author_id            book_title price
0         1           Отцы и дети   450
1         1                 Рудин   300
2         1     Дворянское гнездо   350
3         2      Толстый и тонкий   500
4         2       Дама с собачкой   450
5         3                 Гроза   370
6         3  Таланты и поклонники   290


Получите датафрейм authors_price, соединив датафреймы authors и books по полю author_id.

In [5]:
authors_1 = authors.apply(pd.to_numeric, errors='coerce').fillna(authors)
book_1 = book.apply(pd.to_numeric, errors='coerce').fillna(book)
authors_price = pd.merge(authors_1, book_1, on = 'author_id', how = 'outer')
print(authors_price)

   author_id author_name            book_title  price
0          1    Тургенев           Отцы и дети    450
1          1    Тургенев                 Рудин    300
2          1    Тургенев     Дворянское гнездо    350
3          2       Чехов      Толстый и тонкий    500
4          2       Чехов       Дама с собачкой    450
5          3  Островский                 Гроза    370
6          3  Островский  Таланты и поклонники    290


Создайте датафрейм top5, в котором содержатся строки из authors_price 
с пятью самыми дорогими книгами.

In [6]:
authors_price_top5 = authors_price.head(5)
print(authors_price_top5)

   author_id author_name         book_title  price
0          1    Тургенев        Отцы и дети    450
1          1    Тургенев              Рудин    300
2          1    Тургенев  Дворянское гнездо    350
3          2       Чехов   Толстый и тонкий    500
4          2       Чехов    Дама с собачкой    450


Создайте датафрейм authors_stat на основе информации из authors_price.
В датафрейме authors_stat должны быть четыре столбца:
author_name, min_price, max_price и mean_price,
в которых должны содержаться соответственно имя автора,
минимальная, максимальная и средняя цена на книги этого автора.

In [7]:
authors_price_min = authors_price.groupby('author_name').agg({'price': 'min'})
authors_price_max = authors_price.groupby('author_name').agg({'price': 'max'})
authors_price_mean = authors_price.groupby('author_name').agg({'price': 'mean'})

authors_stat = pd.merge(authors_price_min, authors_price_max, on = 'author_name', how = 'outer')
authors_stat = pd.merge(authors_stat, authors_price_mean, on = 'author_name', how = 'outer')
authors_stat = authors_stat.rename(columns={'price_x':'min_price'})
authors_stat = authors_stat.rename(columns={'price_y':'max_price'})
authors_stat = authors_stat.rename(columns={'price':'mean_price'})
print(authors_stat)


             min_price  max_price  mean_price
author_name                                  
Островский         290        370  330.000000
Тургенев           300        450  366.666667
Чехов              450        500  475.000000


Создайте новый столбец в датафрейме authors_price под названием cover,
в нем будут располагаться данные о том, какая обложка у данной книги - твердая или мягкая.
В этот столбец поместите данные из следующего списка:
['твердая', 'мягкая', 'мягкая', 'твердая', 'твердая', 'мягкая', 'мягкая'].
Просмотрите документацию по функции pd.pivot_table 
с помощью вопросительного знака.
Для каждого автора посчитайте суммарную стоимость книг в твердой и мягкой обложке.
Используйте для этого функцию pd.pivot_table. 
При этом столбцы должны называться "твердая" и "мягкая",
а индексами должны быть фамилии авторов. Пропущенные значения стоимостей заполните нулями,
при необходимости загрузите библиотеку Numpy.
Назовите полученный датасет book_info и сохраните его в формат pickle под названием "book_info.pkl".
Затем загрузите из этого файла датафрейм и назовите его book_info2.
Удостоверьтесь, что датафреймы book_info и book_info2 идентичны.

In [8]:
authors_price['cover'] = pd.Series(['твердая', 'мягкая', 'мягкая', 'твердая', 'твердая', 'мягкая', 'мягкая'])
print(authors_price)

   author_id author_name            book_title  price    cover
0          1    Тургенев           Отцы и дети    450  твердая
1          1    Тургенев                 Рудин    300   мягкая
2          1    Тургенев     Дворянское гнездо    350   мягкая
3          2       Чехов      Толстый и тонкий    500  твердая
4          2       Чехов       Дама с собачкой    450  твердая
5          3  Островский                 Гроза    370   мягкая
6          3  Островский  Таланты и поклонники    290   мягкая


In [9]:
book_info = pd.pivot_table(authors_price, values='price', index=['author_name'], columns=['cover'], aggfunc=np.sum)
print(book_info)

cover        мягкая  твердая
author_name                 
Островский    660.0      NaN
Тургенев      650.0    450.0
Чехов           NaN    950.0


In [13]:
book_info.to_pickle('book_info.pkl')
book_info_2 = pd.read_pickle('book_info.pkl')
print(book_info, "\n\n", book_info_2)

cover        мягкая  твердая
author_name                 
Островский    660.0      NaN
Тургенев      650.0    450.0
Чехов           NaN    950.0 

 cover        мягкая  твердая
author_name                 
Островский    660.0      NaN
Тургенев      650.0    450.0
Чехов           NaN    950.0
